In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [17]:

def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    url="https://www.glassdoor.co.in/Job/india-data-scientist-jobs-SRCH_IL.0,5_IN115_KO6,20.htm"   
    #url= "https://www.glasdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword"+keyword+"&sc.keyword=data+Scientist&locT=&locId=&jobType="    #<-Modification made by Ken Jee

    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)
        time.sleep(.1)
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("react-job-listing")  #looks like this works. Buy since I don't understand what I'm doing....
        for job_button in job_buttons:  
                
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))

            if len(jobs) >= num_jobs:
                break
  
            job_button.click()  #You might 
     
            time.sleep(1)
    
            collected_successfully = False
 
            try:
                driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.    <- Again Ken Jee magic
            except NoSuchElementException:
                pass
            
            while not collected_successfully:

                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "css-1j389vi e1tk4kwz2")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                    
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-1hbqxax e1wijj240"]').text#    <- Again Ken Jee magic
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@class="css-1m5m32b e1tk4kwz4"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            try:
                driver.find_element_by_xpath('.//div[@data-item="tab" and @data-tab-type="overview"]').click()

                try:
                    size = driver.find_element_by_xpath('.//span[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//span[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//span[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//span[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//span[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//span[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//a[@data-test="pagination-next"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame. 




In [18]:
path='C:/Users/hp/Desktop/github/ds-salary/chromedriver'
df=get_jobs('data scientist', 20, False, path,15)
df.to_csv('data_scientist_jobs.csv', index=False)

<ipython-input-17-5e9637bebaa9>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)
<ipython-input-17-5e9637bebaa9>:28: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  job_buttons = driver.find_elements_by_class_name("react-job-listing")  #looks like this works. Buy since I don't understand what I'm doing....


Progress: 0/20


<ipython-input-17-5e9637bebaa9>:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.    <- Again Ken Jee magic
<ipython-input-17-5e9637bebaa9>:50: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
<ipython-input-17-5e9637bebaa9>:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
<ipython-input-17-5e9637bebaa9>:52: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  job_title = driver.find_element_by_xpath('.//div[contains(@class, "css-1j389vi e1tk4kwz2")]').text
<ipython-input-17-5e9637bebaa9>:53: DeprecationWarning: find_element_by_* com

Progress: 1/20
Progress: 2/20
Progress: 3/20
Progress: 4/20
Progress: 5/20
Progress: 6/20
Progress: 7/20
Progress: 8/20
Progress: 9/20
Progress: 10/20
Progress: 11/20
Progress: 12/20
Progress: 13/20
Progress: 14/20
Progress: 15/20
Progress: 16/20
Progress: 17/20
Progress: 18/20
Progress: 19/20
Progress: 20/20


<ipython-input-17-5e9637bebaa9>:148: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('.//a[@data-test="pagination-next"]').click()


In [19]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Product: Data Scientist,Employer Provided Salary:₹34L - ₹51L,Your Job\nWe’re looking for a Data Scientist t...,4.3,Indeed\n4.3,Hyderābād,10000+ Employees,2004,Company - Private,Internet,Information Technology,$2 to $5 billion (USD)
1,Data Scientist (BE/ B-tech 4-6 yrs),Employer Provided Salary:₹25L - ₹35L,CommerceIQ is Hiring Data Scientist (3-5 yrs)\...,4.2,CommerceIQ\n4.2,Bangalore,51 to 200 Employees,2012,Company - Private,Internet,Information Technology,$10 to $25 million (USD)
2,Senior Data Scientist,Employer Provided Salary:₹40L - ₹60L,Your Job\nAs a Data Scientist at Indeed your r...,4.3,Indeed\n4.3,Hyderābād,10000+ Employees,2004,Company - Private,Internet,Information Technology,$2 to $5 billion (USD)
3,Urgent Opening for DATA SCIENTIST ROLE,Employer Provided Salary:₹9L,Experience: Fresher\nLocation:Work from Home t...,-1,iTech Analytic Solutions,Remote,-1,-1,-1,-1,-1,-1
4,Urgent Opening for DATA SCIENTIST ROLE,Employer Provided Salary:₹9L,Experience: Fresher\nLocation:Work from Home t...,-1,iTech Analytic Solutions,Remote,-1,-1,-1,-1,-1,-1
5,Data Scientist,Employer Provided Salary:₹25L,Multiple development and support Data Science ...,4.0,Evoke Technologies\n4.0,Hyderābād,501 to 1000 Employees,2003,Company - Private,IT Service,Information Technology,$25 to $50 million (USD)
6,Data Scientist,Employer Provided Salary:₹30L - ₹45L,Job Description\nRoles and Responsibilities\nJ...,-1,Jubilant FoodWorks,Noida,10000+ Employees,1995,Company - Public,Fast-Food & Quick-Service Restaurant,"Restaurants, Bars & Food Service",Unknown / Non-Applicable
7,Data Scientist,₹8L - ₹10L (Glassdoor Est.),Position Summary\nJob Title— Data Scientist in...,3.9,Deloitte\n3.9,Bangalore,10000+ Employees,1850,Company - Private,Accounting,Accounting & Legal,$10+ billion (USD)
8,Analyst - Data Science - Python/Machine Learning,Employer Provided Salary:₹4L - ₹5L,Analyst - Data Science - Python/Machine Learni...,-1,Next gen computing,New Delhi,-1,-1,-1,-1,-1,-1
9,Data Scientist,-1,Smart Infrastructure Division in Siemens Ltd. ...,4.1,Siemens Limited\n4.1,Goa,5001 to 10000 Employees,2004,Subsidiary or Business Segment,Computer Hardware & Software,Information Technology,$10+ billion (USD)
